In [1]:
from cobra.io import load_matlab_model

from pytfa.io import load_thermoDB,                    \
                        read_lexicon, annotate_from_lexicon,            \
                        read_compartment_data, apply_compartment_data


root_dir = '../../pytfa/'
model_name = root_dir + 'models/'

# Load the model
cobra_model = load_matlab_model(root_dir + 'models/small_ecoli.mat')

In [3]:
import pytfa

In [4]:
import pandas as pd
import numpy as np

In [5]:
def single_coenzyme_transform(model,reaction_id):
    met = {}
    met['nad'] = np.where([x.id == 'nad_c' for x in model.metabolites])[0][0]
    met['nadp'] = np.where([x.id == 'nadp_c' for x in model.metabolites])[0][0]
    met['nadh'] = np.where([x.id == 'nadh_c' for x in model.metabolites])[0][0]
    met['nadph'] = np.where([x.id == 'nadph_c' for x in model.metabolites])[0][0]

    met_objs = {}
    met_objs['nad'] = [x for x in model.metabolites if x.id == 'nad_c'][0]
    met_objs['nadh'] = [x for x in model.metabolites if x.id == 'nadh_c'][0]
    met_objs['nadp'] = [x for x in model.metabolites if x.id == 'nadp_c'][0]
    met_objs['nadph'] = [x for x in model.metabolites if x.id == 'nadph_c'][0]
    
    rxn = [x for x in model.reactions if x.id == reaction_id][0].copy()
    # make a new dictionary with coenzyme swapped
    v = {x:y for x,y in rxn.metabolites.items() if x.id in [x + '_c' for x in list(met)]}
    
    nad_stoich = 0;
    nadh_stoich  = 0;
    
    if len(v) > 0:
        v2 = {}
        for x,y in v.items():
            if x.id == 'nadph_c':
                nadh_stoich = nadh_stoich + y 
                #v2[met_objs['nadh']] = y
            elif x.id == 'nadp_c':
                nad_stoich = nad_stoich + y 
                #v2[met_objs['nad']] = y
            elif x.id == 'nadh_c':
                nadh_stoich = nadh_stoich + y 
                #v2[met_objs['nadph']] = y
            elif x.id == 'nad_c':
                nad_stoich = nad_stoich + y 
                #v2[met_objs['nadp']] = y

        v2[met_objs['nad']] = nad_stoich
        v2[met_objs['nadh']] = nadh_stoich

        rxn.subtract_metabolites(v)
        rxn.add_metabolites(v2)
        rxn.id = rxn.id + '[condensed]'
        model.remove_reactions([x for x in model.reactions if x.id == reaction_id][0])
        model.add_reaction(rxn)
        
    return model

In [6]:
m  = cobra_model.copy()
rxn_ids = [x.id for x in cobra_model.reactions]

In [7]:
m  = cobra_model.copy()
rxn_ids = [x.id for x in cobra_model.reactions]
for rxnid in rxn_ids:
    m = single_coenzyme_transform(m,rxnid)
    
m.remove_reactions([x for x in m.reactions if x.id == 'NADTRHD[condensed]'][0])
m.objective = 'Ec_biomass_iJO1366_WT_53p95M[condensed]'

/Users/joshuagoldford/opt/miniconda3/envs/coenzymes/lib/python3.8/site-packages/cobra/core/model.py:745: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [105]:

# Load reaction DB
thermo_data = load_thermoDB(root_dir + 'data/thermo_data.thermodb')
lexicon = read_lexicon(root_dir + 'models/small_ecoli/lexicon.csv')
compartment_data = read_compartment_data(root_dir + 'models/small_ecoli/compartment_data.json')

# Initialize the model
tmodel = pytfa.ThermoModel(thermo_data, m)
tmodel.name = 'NAD_only'

# Annotate the model
annotate_from_lexicon(tmodel, lexicon)
apply_compartment_data(tmodel, compartment_data)

## TFA conversion
tmodel.prepare()
tmodel.convert()

2021-01-31 13:06:15,782 - thermomodel_None - INFO - # Model initialized with units kcal/mol and temperature 298.15 K
2021-01-31 13:06:15,786 - thermomodel_None - INFO - # Model preparation starting...
2021-01-31 13:06:15,941 - thermomodel_None - WARNING - Warning : NULL/U
2021-01-31 13:06:15,942 - thermomodel_None - WARNING - Warning : NULL/L
2021-01-31 13:06:15,943 - thermomodel_None - WARNING - Warning : NULL/L
2021-01-31 13:06:15,944 - thermomodel_None - WARNING - Warning : NULL/U
2021-01-31 13:06:15,947 - thermomodel_None - WARNING - Warning : NULL/L
2021-01-31 13:06:15,948 - thermomodel_None - WARNING - Warning : NULL/L
2021-01-31 13:06:16,007 - thermomodel_None - WARNING - Warning : NULL/U
2021-01-31 13:06:16,012 - thermomodel_None - WARNING - Warning : NULL/L
2021-01-31 13:06:16,016 - thermomodel_None - WARNING - Warning : NULL/L
2021-01-31 13:06:16,018 - thermomodel_None - WARNING - Warning : NULL/U
2021-01-31 13:06:16,019 - thermomodel_None - WARNING - Warning : NULL/L
2021-01

In [107]:
## Info on the model
tmodel.print_info()

                    value
key                      
name             NAD_only
description      NAD_only
num constraints      3337
num variables        3472
num metabolites       304
num reactions         598
                             value
key                               
num metabolites(thermo)        254
num reactions(thermo)          413
pct metabolites(thermo)  83.552632
pct reactions(thermo)    69.063545


In [108]:
## Optimality
tfa_solution = tmodel.optimize()

In [121]:
m2 = cobra_model.copy()

In [122]:
m2.medium['EX_O']

{}

In [130]:
m2.reactions[6]

Reaction identifier,DM_ac_e
Name,
Memory address,0x07f9df2333e80
Stoichiometry,ac_e --> Acetate -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [131]:
m2.medium['DM_ac_e'] = 1000

In [151]:
m2.reactions.get_by_id('DM_glc_e').lower_bound = 0
m2.reactions.get_by_id('DM_glycogenn1_c').lower_bound = -1
m2.reactions.get_by_id('DM_ac_e').lower_bound = -10

In [152]:
m2.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,DM_ac_e,10,2,95.40%
ca2_e,DM_ca2_e,0.001285,0,0.00%
cbl1_e,DM_cbl1_e,5.785E-05,62,0.02%
cl_e,DM_cl_e,0.001285,0,0.00%
cobalt2_e,DM_cobalt2_e,6.226E-06,0,0.00%
cu2_e,DM_cu2_e,0.0001748,0,0.00%
fe2_e,DM_fe2_e,0.002101,0,0.00%
fe3_e,DM_fe3_e,0.001927,0,0.00%
glycogenn1_c,DM_glycogenn1_c,0.04,24,4.58%
h_e,DM_h_e,7.63,0,0.00%


In [157]:
media = pd.read_csv('../assets/media.C180N93E7.txt',sep='\t',index_col=0)

In [165]:
reaction_ids = [x.id for x in m2.reactions]

In [170]:
exchange_rxns = [x for x in  reaction_ids if x.split('_')[0] == 'DM']

In [227]:
s  =cobra_model.optimize()
fs = s[s.fluxes < 0].index.get_level_values(0).tolist()

In [230]:
mets = [x for x in fs if x in exchange_rxns]

In [231]:
mets

['DM_ca2_e',
 'DM_cbl1_e',
 'DM_cl_e',
 'DM_cobalt2_e',
 'DM_cu2_e',
 'DM_fe2_e',
 'DM_fe3_e',
 'DM_glc_e',
 'DM_glycogenn1_c',
 'DM_k_e',
 'DM_mg2_e',
 'DM_mn2_e',
 'DM_mobd_e',
 'DM_nh4_e',
 'DM_ni2_e',
 'DM_o2_e',
 'DM_pi_e',
 'DM_so4_e',
 'DM_zn2_e']

In [186]:
pd.DataFrame({'rn_ids': exchange_rxns}).to_csv('../assets/exchangeRxns.reducedModel.31Jan2021.csv')

In [233]:
erns = pd.read_csv('../assets/exchangeRxns.reducedModel.31Jan2021.csv').fillna(False)

In [272]:
csources = erns[erns['carbon'] == True].rn_ids.unique().tolist() + ['None']
nsources = erns[erns['nitrogen'] == True].rn_ids.unique().tolist() + ['None']
esources = erns[erns['electronAcceptor'] == True].rn_ids.unique().tolist() + ['None']
ssources = erns[erns['sulfer'] == True].rn_ids.unique().tolist() + ['None']
psources = erns[erns['phosphorous'] == True].rn_ids.unique().tolist() + ['None']


In [242]:
essential_reactions = [x for x in mets if x not in (csources+nsources+esources+ssources+psources)]

In [243]:
pd.DataFrame({'rn_ids': essential_reactions}).to_csv('../assets/exchangeRxns.reducedModel.Essential.31Jan2021.csv')

In [263]:
#essential_reactions

In [252]:
h = {x: cobra_model.reactions.get_by_id(x).lower_bound for x in essential_reactions}

In [264]:
#pd.DataFrame(pd.Series(h),columns =['lower_bound']).to_csv('../assets/exchangeRxns.reducedModel.Essential.lowerBounds.31Jan2021.csv)

In [260]:
g = pd.DataFrame(pd.Series(h),columns =['lower_bound'])

In [261]:
g.to_csv('../assets/exchangeRxns.reducedModel.Essential.lowerBounds.31Jan2021.csv')

In [273]:
media_sets = {'carbon': [], 'nitrogen': [], 'electron': [], 'sulfur': [] , 'phosphorous':[] }
for c in csources:
    for n in nsources:
        for e in esources:
            for s in ssources:
                for p in psources:
                    media_sets['carbon'].append(c)
                    media_sets['nitrogen'].append(n)
                    media_sets['electron'].append(e)
                    media_sets['sulfur'].append(s)
                    media_sets['phosphorous'].append(p)
                    
media_sets = pd.DataFrame(media_sets)

In [275]:
media_sets.to_csv('../assets/reducedModel.mediaSets.31Jan2021.csv')

In [276]:
media_sets

,carbon,nitrogen,electron,sulfur,phosphorous
0,DM_5DRIB,DM_5DRIB,DM_o2_e,DM_5mtr_e,DM_pi_e
1,DM_5DRIB,DM_5DRIB,DM_o2_e,DM_5mtr_e,None
2,DM_5DRIB,DM_5DRIB,DM_o2_e,DM_cys-L_e,DM_pi_e
3,DM_5DRIB,DM_5DRIB,DM_o2_e,DM_cys-L_e,None
4,DM_5DRIB,DM_5DRIB,DM_o2_e,DM_gthrd_e,DM_pi_e
...,...,...,...,...,...
22355,None,None,None,DM_gthrd_e,None
22356,None,None,None,DM_so4_e,DM_pi_e
22357,None,None,None,DM_so4_e,None
22358,None,None,None,None,DM_pi_e
